In [70]:
import torch
import os
# move to the root directory
#os.chdir("../MemFlow/")
from core.Networks import build_network
from configs.kitti_memflownet import get_cfg

In [71]:
print(torch.__file__)

/home/pavan/SRA/.venv/lib/python3.11/site-packages/torch/__init__.py


In [72]:
cfg = get_cfg()
model = build_network(cfg)

MemFlowNet_skflow
[Using basicencoder as context encoder]
[Using basicencoder as feature encoder]
[Using GMA-SK2]
[Using corr_fn default]


In [73]:
kitti_weights = "/home/pavan/MemFlow/Weights/Memflow weights/MemFlowNet_kitti.pth"
model.load_state_dict(torch.load(kitti_weights)['model'])

<All keys matched successfully>

In [74]:
model.cnet.__sizeof__()

24

In [75]:
model.fnet.__sizeof__()

24

In [76]:
# random tensor of shape 3x3
x = torch.rand(2,2,2,2, 2)
print(x)
x.flatten(0,1)




tensor([[[[[0.6593, 0.6992],
           [0.7205, 0.1069]],

          [[0.2074, 0.9261],
           [0.9792, 0.0663]]],


         [[[0.1914, 0.8234],
           [0.0649, 0.3367]],

          [[0.7605, 0.2909],
           [0.9313, 0.4216]]]],



        [[[[0.2341, 0.0851],
           [0.8776, 0.5528]],

          [[0.6996, 0.0929],
           [0.5199, 0.4828]]],


         [[[0.8608, 0.4582],
           [0.4901, 0.6517]],

          [[0.0937, 0.0750],
           [0.5717, 0.8803]]]]])


tensor([[[[0.6593, 0.6992],
          [0.7205, 0.1069]],

         [[0.2074, 0.9261],
          [0.9792, 0.0663]]],


        [[[0.1914, 0.8234],
          [0.0649, 0.3367]],

         [[0.7605, 0.2909],
          [0.9313, 0.4216]]],


        [[[0.2341, 0.0851],
          [0.8776, 0.5528]],

         [[0.6996, 0.0929],
          [0.5199, 0.4828]]],


        [[[0.8608, 0.4582],
          [0.4901, 0.6517]],

         [[0.0937, 0.0750],
          [0.5717, 0.8803]]]])

In [77]:
model

MemFlowNet(
  (cnet): BasicEncoder(
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (relu1): ReLU(inplace=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

Trying with Demo Image from the repo

In [80]:
input_image1 = "/home/pavan/MemFlow/demo_input_images/frame_0001.png"
input_image2 = "/home/pavan/MemFlow/demo_input_images/frame_0002.png"
input_image3 = "/home/pavan/MemFlow/demo_input_images/frame_0003.png"

input_image4 = "/home/pavan/MemFlow/demo_input_images/frame_0004.png"
input_image5 = "/home/pavan/MemFlow/demo_input_images/frame_0005.png"
input_image6 = "/home/pavan/MemFlow/demo_input_images/frame_0006.png"

import cv2
import numpy as np
import matplotlib.pyplot as plt



In [116]:
""" img1 = cv2.imread(input_image1)
img2 = cv2.imread(input_image2)
img3 = cv2.imread(input_image3)

img4 = cv2.imread(input_image4)
img5 = cv2.imread(input_image5)
img6 = cv2.imread(input_image6)
 """

# REplace img 1 to 6 with dummy array of shape 224x224x3
img1 = np.random.rand(224,224,3)
img2 = np.random.rand(224,224,3)
img3 = np.random.rand(224,224,3)

img4 = np.random.rand(224,224,3)
img5 = np.random.rand(224,224,3)
img6 = np.random.rand(224,224,3)

In [123]:
# stack the 3 images as tensors
input_frame1 = np.stack([img1, img2, img3], axis=0)
input_frame2 = np.stack([img4, img5, img6], axis=0)

input_frame1 = torch.tensor(input_frame1).permute(0,3,1,2).float()
input_frame2 = torch.tensor(input_frame2).permute(0,3,1,2).float()

# stack the 2 frames as tensors
input_frames = torch.stack([input_frame1, input_frame2], dim=0)
input_frames = input_frames.cuda()


In [124]:
model = model.cuda()

In [125]:
coords0, coords1, fmaps = model.encode_features(input_frames,)

In [126]:
query, key, net, inp = model.encode_context(input_frames)

In [127]:
query.shape, key.shape, net.shape, inp.shape

(torch.Size([2, 128, 3, 28, 28]),
 torch.Size([2, 128, 3, 28, 28]),
 torch.Size([2, 3, 128, 28, 28]),
 torch.Size([2, 3, 128, 28, 28]))

In [128]:
flow_prediction, current_values = model.predict_flow(net, inp, coords0, coords1, 
                                                     fmaps, query, key, ref_values=None )

RuntimeError: FlashAttention only supports Ampere GPUs or newer.